# CCRB ChatBot Notebook

## Goal:

1. To make a tool that reads a CSV of the CCRB database
2. Adds the data to an SQL database
3. Uses an LLM to convert human questions into SQL queries
4. Uses a SQL platform like Postgres to perform the query 
5. Uses an LLM to make the returned answer into a human readable response and presents it to the user

### Load CSV

In [ ]:
import pandas as pd
import sqlalchemy as sa


In [2]:
df = pd.read_csv("complaintclean.csv")

/var/folders/xv/zb6mjv_93435jvb4s_wsh3xm0000gn/T/ipykernel_93420/630838838.py:1: DtypeWarning: Columns (11,14,16,17,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("complaintclean.csv")


In [3]:
df.head()

,complaintid,shieldno,lastname,firstname,lastactivedate,daysonforce,officerrace_cat,officergender_cat,incidentranklong_cat,currentranklong_cat,...,locationtype_cat,contactreason_cat,contactoutcome_cat,impactedrace_cat,impactedgender_cat,ccrbdisposition_cat,boardcat_cat,penaltyrec_cat,subst,charges
0,202101017,4419,Ruscillo,Taylor,04/01/21,672,White,Female,Police Officer,Police Officer,...,Apartment/house,Report-domestic dispute,No arrest made or summons issued,Hispanic,Male,Complainant Unavailable,Truncated,NaN,0,NaN
1,202100294,22851,Medina,Nelson,04/01/21,4753,Hispanic,Male,Police Officer,Police Officer,...,Subway station/train,C/V intervened on behalf of/observed encounter...,Arrest - other violation/crime,Black,Male,Closed - Pending Litigation,Closed - Pending Litigation,NaN,0,NaN
2,202100294,14350,Pierre,Bebeto,04/01/21,924,Black,Male,Police Officer,Police Officer,...,Subway station/train,C/V intervened on behalf of/observed encounter...,Arrest - other violation/crime,Black,Male,Closed - Pending Litigation,Closed - Pending Litigation,NaN,0,NaN
3,202100229,29706,Chapman,John,04/01/21,361,White,Male,Police Officer,Police Officer,...,Apartment/house,Report-domestic dispute,No arrest made or summons issued,NaN,Male,Complaint Withdrawn,Truncated,NaN,0,NaN
4,202100229,13834,Tafuro,Justin,04/01/21,2733,White,Male,Police Officer,Police Officer,...,Apartment/house,Report-domestic dispute,No arrest made or summons issued,NaN,Male,Complaint Withdrawn,Truncated,NaN,0,NaN


In [7]:
df.columns

Index(['complaintid', 'shieldno', 'lastname', 'firstname', 'lastactivedate',
       'daysonforce', 'officerrace_cat', 'officergender_cat',
       'incidentranklong_cat', 'currentranklong_cat', 'status_cat',
       'incidentdate', 'fadotype_cat', 'allegation_cat', 'locationtype_cat',
       'contactreason_cat', 'contactoutcome_cat', 'impactedrace_cat',
       'impactedgender_cat', 'ccrbdisposition_cat', 'boardcat_cat',
       'penaltyrec_cat', 'subst', 'charges'],
      dtype='object')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279406 entries, 0 to 279405
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   complaintid           279406 non-null  int64  
 1   shieldno              279406 non-null  int64  
 2   lastname              279406 non-null  object 
 3   firstname             279406 non-null  object 
 4   lastactivedate        279406 non-null  object 
 5   daysonforce           279406 non-null  int64  
 6   officerrace_cat       279406 non-null  object 
 7   officergender_cat     279406 non-null  object 
 8   incidentranklong_cat  259193 non-null  object 
 9   currentranklong_cat   279406 non-null  object 
 10  status_cat            279406 non-null  object 
 11  incidentdate          259292 non-null  object 
 12  fadotype_cat          279406 non-null  object 
 13  allegation_cat        279021 non-null  object 
 14  locationtype_cat      179158 non-null  object 
 15  

In [15]:
df.iloc[0]

complaintid                                    202101017
shieldno                                            4419
lastname                                        Ruscillo
firstname                                         Taylor
lastactivedate                                  04/01/21
daysonforce                                          672
officerrace_cat                                    White
officergender_cat                                 Female
incidentranklong_cat                      Police Officer
currentranklong_cat                       Police Officer
status_cat                                        Active
incidentdate                                    02/15/21
fadotype_cat                          Abuse Of Authority
allegation_cat                          Threat of arrest
locationtype_cat                         Apartment/house
contactreason_cat                Report-domestic dispute
contactoutcome_cat      No arrest made or summons issued
impactedrace_cat               